In [281]:
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
import requests
import io
import tabula
import pandas as pd
import numpy as np

In [ ]:
BASE_URL = "https://dijlovasok.hu/index.php/programgyujt"

In [ ]:
def get_contents_of_page (url, headers = None) :
    req = requests.get(url, headers = headers if headers != None else {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"
    })
    
    return req.content

In [ ]:
def soupify (content) :
    return BeautifulSoup(content, "html.parser")

In [ ]:
def get_links_from_main_page ():
    soup = soupify(get_contents_of_page(BASE_URL))
    # print(soup.prettify())
    program_hrefs = list(
        set(
            map(
                lambda href: href["href"], 
                filter(lambda href: href["href"].startswith("/index.php/programgyujt/"), 
                    soup.find_all("a", href = True)
                )
            )
        )
    )
    return program_hrefs    

In [ ]:
def get_pdf_links_from_page (url):
    soup: BeautifulSoup = soupify(get_contents_of_page(url))
    pdf_hrefs = list(
            set(
                map(
                    lambda href: href["href"], 
                    filter(lambda href: href["href"].endswith(".pdf"), 
                        soup.find_all("a", href = True)
                    )
                )
            )
        )
    return pdf_hrefs

In [220]:
def get_pdf_dataframes () :

    pdf_texts = {}

    for page_url in get_links_from_main_page() :
        
        print(f"Scraping page {page_url}...")
        
        page_content = {}
        
        for pdf_url in get_pdf_links_from_page(BASE_URL + page_url) :
            
            print(f"Downloading pdf {pdf_url}...")
                        
            user_agent= 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'

            dfs = tabula.read_pdf(
                pdf_url, 
                multiple_tables=True, 
                pages="all", 
                guess=True,
                encoding="cp1252",
                user_agent=user_agent,
                # output_format="dataframe",
                pandas_options={
                    # "names": ["ID", "Position", "Description", "Notes"]
                    "header": 0
                },
                lattice=True,
                silent=True
            )   
                
            if len(dfs) == 0 :
                continue
                
            page_content[pdf_url] = dfs
            
        pdf_texts[page_url] = page_content
        
    return pdf_texts

In [221]:
database = get_pdf_dataframes()

Scraping page /index.php/programgyujt/1457-kozep-osztalyu-feladatok-mb-program...
Scraping page /index.php/programgyujt/1031-elokeszito-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1217-vezetoszaras-feladat...
Scraping page /index.php/programgyujt/1035-kozep-osztalyu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1008-fiatal-lovak-programjai...
Scraping page /index.php/programgyujt/1034-konnyu-kozep-osztalyu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1037-nagydij-szintu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1033-konnyu-osztalyu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1032-alapfoku-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1036-nehez-osztalyu-feladatok-hatalyos-2018-05-01...


In [282]:
def inspect_database(database) :
    for page_url in database:
        page = database[page_url]
        for program_url in page:
            program = page[program_url]
            for df in program:
                yield df

In [283]:
next(inspect_database(database))

Empty DataFrame
Columns: [Unnamed: 0]
Index: []

In [259]:
list(list(database.values())[0].values())[3][1]

Unnamed: 0       Unnamed: 1  \
0          1.0      A\r\rX\r\rC   
1          2.0              H-E   
2          3.0         E-X\rX-B   
3          4.0              B-F   
4          5.0             A\rL   
5          6.0              L-R   
6          7.0                C   
7          8.0           H-K\rK   
8          9.0             A\rL   
9         10.0              L-S   
10        11.0  S-H-C-M\rM-K\rK   
11        12.0              NaN   
12        13.0                A   
13        14.0           F-E\rE   
14        15.0    S\rI-R között   
15        16.0       I-S között   
16        17.0      (E-S-R-S-I)   

                                              Feladat  Unnamed: 2  Unnamed: 3  \
0   Belovaglás összeszedett\rügetésben\rÁllj, kösz...         NaN         NaN   
1                                           Vállat be         NaN         NaN   
2       Fél kiskör balra, 10m\rFél kiskör jobbra, 10m         NaN         NaN   
3                                           Vállat be         NaN         NaN   
4                        Középbõl\rKiskör jobbra, 10m         NaN         NaN   
5                                   Jobbra oldaljárás         NaN         NaN   
6   Állj, 5 lépést hátra lépj, abból\rösszeszedett...         NaN         NaN   
7                    Középügetés\rÖsszeszedett ügetés         NaN         NaN   
8                         Középbõl\rKiskör balra, 10m         NaN         NaN   
9                                    Balra oldaljárás         NaN         NaN   
10  Összeszedett ügetés\rÁtlóváltás, középügetés\r...         NaN         NaN   
11  Átmenetek összeszedett\rügetésbõl\rközépügetés...         NaN         NaN   
12                                         Középlépés         NaN         NaN   
13          Félátlóváltás, nyújtott lépés\rKözéplépés         NaN         NaN   
14          Fordulat jobbra\rRöviden hátra arc jobbra         NaN         NaN   
15                            Röviden hátra arc balra         NaN         NaN   
16                                     (A középlépés)         NaN         NaN   

    Unnamed: 4  Unnamed: 5  Unnamed: 6  Megjegyzés  
0          NaN         NaN         NaN         NaN  
1          NaN         NaN         NaN         NaN  
2          NaN         NaN         NaN         NaN  
3          NaN         NaN         NaN         NaN  
4          NaN         NaN         NaN         NaN  
5          NaN         2.0         NaN         NaN  
6          NaN         NaN         NaN         NaN  
7          NaN         NaN         NaN         NaN  
8          NaN         NaN         NaN         NaN  
9          NaN         2.0         NaN         NaN  
10         NaN         NaN         NaN         NaN  
11         NaN         NaN         NaN         NaN  
12         NaN         NaN         NaN         NaN  
13         NaN         NaN         NaN         NaN  
14         NaN         2.0         NaN         NaN  
15         NaN         2.0         NaN         NaN  
16         NaN         NaN         NaN         NaN

In [418]:
from pandas import DataFrame
from typing import List

def convert_program_dfs_to_useful_df (dfs: List[DataFrame]) :
            
    dfs = list(filter(lambda df: len(df) != 0, dfs))
    
    if len(dfs) == 0 :
        return None
    
    if len(dfs) == 1 :
        if len(dfs[0]) != 0 :
            return dfs[0]
        else :
            return None
    
    # joining
    
    result = dfs[0]
    
    column_num = len(result.columns)
    
    for df in dfs[1:] :
        
        if len(df.columns) == column_num :
            result = pd.concat([result, df], ignore_index = True)
            continue
        
        if len(df.columns) == column_num + 2 :
            
            columns = df.columns
            
            df = df.drop(columns[0], axis=1)
            df = df.drop(columns[-1], axis=1)
            
            # rename unnamed columns
            
            def column_renamer(column):
                if str(column).startswith("Unnamed") == False :
                    return column
                num = int(column.split(" ")[1]) - 1
                print(f"{column} --> Unnamed: {str(num)}")
                return "Unnamed: " + str(num)
            
            df = df.rename(column_renamer, axis=1)
            
            result = pd.concat([result, df], ignore_index = True)
            continue
        
        # raise Exception(f"WARNING: df column width mismatch: cannot append {len(df.columns)} to {column_num}!")
        
    return result

In [419]:

def convert_database_to_useful_dfs (database) :
    for page in database.values() :
        for program_url in page :
            program = page[program_url]
            try :
                yield convert_program_dfs_to_useful_df(program)
            except Exception as e :
                print(f"URL in question: {program_url}")
                raise e

In [420]:
iter = convert_database_to_useful_dfs(database)

In [459]:
next(iter)

Unnamed: 1 --> Unnamed: 0
Unnamed: 2 --> Unnamed: 1
Unnamed: 3 --> Unnamed: 2
Unnamed: 4 --> Unnamed: 3
Unnamed: 5 --> Unnamed: 4
Unnamed: 6 --> Unnamed: 5
Unnamed: 7 --> Unnamed: 6


Unnamed: 0          Unnamed: 1  \
0          1.0      A\r\rX\r\r\rXC   
1          2.0           C\rMXK\rK   
2          3.0                  KA   
3          4.0               A\rDE   
4          5.0                   E   
5          6.0              EH\rHC   
6          7.0            CM\rM\rH   
7          8.0                 HSP   
8          9.0  P-F között\r\r\rFA   
9         10.0               A\rDB   
10        11.0                   B   
11        12.0              BM\rMC   
12        13.0             C\r\rCH   
13        14.0             HK\rKAF   
14        15.0  F(X)\r\rF-X között   
15        16.0              F\rFAK   
16        17.0         KX\rX\rI\rC   
17        18.0          HP\r\rPFAK   
18        19.0  K(X)\r\rK-X között   
19        20.0              K\rKAF   
20        21.0         FX\rX\rI\rC   
21        22.0                 MXK   
22        23.0                   K   
23        24.0                A\rX   
24         NaN                 NaN   
25         NaN                 NaN   
26         NaN                 NaN   
27         NaN                 NaN   
28         NaN                 NaN   

                                              Feladat  \
0   Belovaglás összeszedett\rvágtában\rÁll-mozdula...   
1   Fordulat jobbra\rNyújtott ügetés\rÖsszeszedett...   
2       Átmenetek M-nél és K-nál\rÖsszeszedett ügetés   
3         Fordulat a középvonalra\rFéloldalazás balra   
4                               Kiskör jobbra (8 m ?)   
5               Vállat be jobbra\rÖsszeszedett ügetés   
6   Összeszedett lépés\rFordulat jobbra\rFordulat ...   
7                                      Nyújtott lépés   
8   Összeszedett lépés és\rmegindulás összeszedett...   
9        Fordulat a középvonalra\rFéloldalazás jobbra   
10                               Kiskör balra (8 m ?)   
11               Vállat be balra\rÖsszeszedett ügetés   
12  Megindulás összeszedett\rvágtában balra\rÖssze...   
13                     Középvágta\rÖsszeszedett vágta   
14  Megindulás X felé összeszedett\rvágtában\rFélp...   
15                    Ugrásváltás\rÖsszeszedett vágta   
16  Féloldalazás jobbra\rFordulat a középvonalra\r...   
17  Átlóvonalon 3 ugrásváltás\rminden 4. vágtaugrá...   
18  Megindulás X felé összeszedett\rvágtában\rFélp...   
19                    Ugrásváltás\rÖsszeszedett vágta   
20  Féloldalazás balra\rFordulat a középvonalra\rU...   
21                                     Nyújtott vágta   
22                 Összeszedett vágta és\rugrásváltás   
23  Fordulat a középvonalra\rÁll-mozdulatlanság-kö...   
24  A négyszöget A-nál lépésben,\rhosszú száron el...   
25                                                NaN   
26                                                NaN   
27                                                NaN   
28                                                NaN   

                   Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  \
0                        10.0         NaN         NaN         NaN         NaN   
1                        10.0         NaN         NaN         NaN         NaN   
2                        10.0         NaN         NaN         NaN         NaN   
3                        10.0         NaN         NaN         NaN         NaN   
4                        10.0         NaN         NaN         NaN         NaN   
5                        10.0         NaN         NaN         NaN         NaN   
6                        10.0         NaN         NaN         2.0         NaN   
7                        10.0         NaN         NaN         2.0         NaN   
8                        10.0         NaN         NaN         NaN         NaN   
9                        10.0         NaN         NaN         NaN         NaN   
10                       10.0         NaN         NaN         NaN         NaN   
11                       10.0         NaN         NaN         NaN         NaN   
12                       10.0         NaN         NaN         NaN         NaN   
13                